**The goal here is to explore & cluster neighborhoods in Toronto**

In [68]:
import pandas as pd
from bs4 import BeautifulSoup as bs
import numpy as np
import requests
from pathlib import Path
from geopy.geocoders import Nominatim
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
from sklearn.cluster import KMeans

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage

!conda install -c conda-forge folium=0.5.0 --yes
import folium 

print('Libraries imported.')

Fetching package metadata .............
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /opt/conda/envs/DSX-Python35:
#
folium                    0.5.0                      py_0    conda-forge
Libraries imported.


**Load the pandas dataframe (created in part 2 of the assignment)**

In [69]:
df_toronto = pd.read_csv('toronto_base.csv')
df_toronto.drop(['Unnamed: 0'], axis = 1, inplace = True)
df_toronto.head(7)

,Postalcode,Borough,Neighbourhood,Postal Code,Latitude,Longitude
0,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",M1B,43.806686,-79.194353
1,M5E,Downtown Toronto,Berczy Park,M1C,43.784535,-79.160497
2,M6B,North York,Glencairn,M1E,43.763573,-79.188711
3,M6H,West Toronto,"Dovercourt Village, Dufferin",M1G,43.770992,-79.216917
4,M4P,Central Toronto,Davisville North,M1H,43.773136,-79.239476
5,M4M,East Toronto,Studio District,M1J,43.744734,-79.239476
6,M2L,North York,"Silver Hills, York Mills",M1K,43.727929,-79.262029


In [70]:
df_toronto = df_toronto[df_toronto['Neighbourhood'].str.contains('Toronto')] #Use a mask to filter out only neighborhoods that include Toronto
df_toronto

,Postalcode,Borough,Neighbourhood,Postal Code,Latitude,Longitude
19,M4J,East York,East Toronto,M2K,43.786947,-79.385975
33,M4R,Central Toronto,North Toronto West,M3N,43.761631,-79.520999
45,M5J,Downtown Toronto,"Harbourfront East, Toronto Islands, Union Station",M4P,43.712751,-79.390197
64,M5K,Downtown Toronto,"Design Exchange, Toronto Dominion Centre",M5P,43.696948,-79.411307
68,M3K,North York,"CFB Toronto, Downsview East",M5V,43.628947,-79.394420
73,M8V,Etobicoke,"Humber Bay Shores, Mimico South, New Toronto",M6C,43.693781,-79.428191
87,M5S,Downtown Toronto,"Harbord, University of Toronto",M7Y,43.662744,-79.321558


In [71]:
# create map of Toronto using latitude and longitude values
latitude = 43.6532
longitude = -79.3832

map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighbourhood in zip(df_toronto['Latitude'], df_toronto['Longitude'], df_toronto['Borough'], df_toronto['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [72]:
# @hidden cell
CLIENT_ID = 'MK51CHNXA04S2LWHNEVGIAIC43GBF1CPZKPPDVNLY35XFGLJ' # your Foursquare ID
CLIENT_SECRET = 'JHFP3TDXYXYBIABED0UBWA53WRGDTO5IYGY0GZTTZBW33QK0' # No peeking at my secret key
VERSION = '20190101' # Foursquare API version

In [73]:
import json
def getNearbyVenues(names, latitudes, longitudes, radius=500): #This function was built by the Coursera lab
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [74]:
#Use the function to pull venues near each neighborhood
LIMIT = 500
toronto_venues = getNearbyVenues(names=df_toronto['Neighbourhood'],
                                   latitudes=df_toronto['Latitude'],
                                   longitudes=df_toronto['Longitude']
                                  )

East Toronto
North Toronto West
Harbourfront East, Toronto Islands, Union Station
Design Exchange, Toronto Dominion Centre
CFB Toronto, Downsview East
Humber Bay Shores, Mimico South, New Toronto
Harbord, University of Toronto


In [75]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 47 uniques categories.


In [76]:
#Start the clustering analysis with one-hot encoding for pre-processing
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.shape

(53, 48)

In [77]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,Auto Workshop,Bank,Boat or Ferry,...,Pizza Place,Plane,Restaurant,Sandwich Place,Sculpture Garden,Skate Park,Smoke Shop,Spa,Sushi Restaurant,Trail
0,"CFB Toronto, Downsview East",0.071429,0.071429,0.071429,0.142857,0.142857,0.142857,0.000000,0.00,0.071429,...,0.000000,0.071429,0.000000,0.000,0.071429,0.000000,0.000000,0.000000,0.00,0.00
1,"Design Exchange, Toronto Dominion Centre",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,...,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.25,0.25
2,East Toronto,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.25,0.000000,...,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.00,0.00
3,"Harbord, University of Toronto",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.066667,0.00,0.000000,...,0.066667,0.000000,0.066667,0.000,0.000000,0.066667,0.066667,0.066667,0.00,0.00
4,"Harbourfront East, Toronto Islands, Union Station",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,...,0.000000,0.000000,0.000000,0.125,0.000000,0.000000,0.000000,0.000000,0.00,0.00
5,"Humber Bay Shores, Mimico South, New Toronto",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,...,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.00,0.25
6,North Toronto West,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,...,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.00,0.00


In [78]:
#Show the top 5 most frequent venues in our neighborhood analysis
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----CFB Toronto, Downsview East----
              venue  freq
0    Airport Lounge  0.14
1   Airport Service  0.14
2  Airport Terminal  0.14
3           Airport  0.07
4          Boutique  0.07


----Design Exchange, Toronto Dominion Centre----
                venue  freq
0               Trail  0.25
1  Mexican Restaurant  0.25
2    Sushi Restaurant  0.25
3       Jewelry Store  0.25
4                 Gym  0.00


----East Toronto----
                 venue  freq
0  Japanese Restaurant  0.25
1                 Bank  0.25
2   Chinese Restaurant  0.25
3                 Café  0.25
4              Airport  0.00


----Harbord, University of Toronto----
                  venue  freq
0                Garden  0.07
1               Brewery  0.07
2         Garden Center  0.07
3  Fast Food Restaurant  0.07
4        Farmers Market  0.07


----Harbourfront East, Toronto Islands, Union Station----
               venue  freq
0       Burger Joint  0.12
1  Food & Drink Shop  0.12
2              Hotel  0.12
3  

In [79]:
#Create a function that sorts the venues by frequency
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [80]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.shape

(7, 11)

In [81]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:103]

array([1, 0, 4, 1, 1, 3, 2], dtype=int32)

In [82]:
toronto_merged = df_toronto

# add clustering labels
toronto_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')

toronto_merged.head() 

,Postalcode,Borough,Neighbourhood,Postal Code,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
19,M4J,East York,East Toronto,M2K,43.786947,-79.385975,1,Chinese Restaurant,Bank,Café,Japanese Restaurant,Trail,Burger Joint,Fast Food Restaurant,Farmers Market,Discount Store,Comic Shop
33,M4R,Central Toronto,North Toronto West,M3N,43.761631,-79.520999,0,Liquor Store,Discount Store,Grocery Store,Gym / Fitness Center,Brewery,Farmers Market,Comic Shop,Clothing Store,Chinese Restaurant,Café
45,M5J,Downtown Toronto,"Harbourfront East, Toronto Islands, Union Station",M4P,43.712751,-79.390197,4,Breakfast Spot,Food & Drink Shop,Clothing Store,Sandwich Place,Gym,Park,Hotel,Burger Joint,Brewery,Discount Store
64,M5K,Downtown Toronto,"Design Exchange, Toronto Dominion Centre",M5P,43.696948,-79.411307,1,Trail,Sushi Restaurant,Mexican Restaurant,Jewelry Store,Brewery,Farmers Market,Discount Store,Comic Shop,Clothing Store,Chinese Restaurant
68,M3K,North York,"CFB Toronto, Downsview East",M5V,43.628947,-79.394420,1,Airport Terminal,Airport Lounge,Airport Service,Airport,Harbor / Marina,Boat or Ferry,Plane,Boutique,Sculpture Garden,Airport Gate


In [83]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters